In [1]:
from prophet import Prophet
import numpy as np
import pandas as pd
import json
import warnings

In [2]:
## 데이터로드
df = pd.read_csv("C:/Users/User/github/Quant/data/kospi_category.csv", encoding='cp949')

In [5]:
start_date = '2004-01-01'
end_date = '2020-03-01'
 
df.index = pd.date_range(start_date,end_date,freq='m')

In [6]:
df.drop('date',axis=1, inplace=True)

In [20]:
len(df)*0.7

135.79999999999998

In [22]:
train = df.iloc[:135,:]
test = df.iloc[135:,:]

In [24]:
train.corr()

,제조업,음식료품,섬유의복,종이목재,화학,의약품,비금속광물,철강금속,기계,전기전자,...,유통업,전기가스업,건설업,운수창고업,통신업,금융업,은행,증권,보험,서비스업
제조업,1.000000,0.852713,0.796107,0.350777,0.956254,0.711579,0.009088,0.755305,0.461514,0.899321,...,0.844306,0.298498,0.148679,0.627703,-0.144394,0.355066,0.195860,0.155999,0.769443,0.966039
음식료품,0.852713,1.000000,0.896869,0.514889,0.720400,0.780014,0.310595,0.479571,0.249604,0.892836,...,0.757608,0.572289,0.056211,0.413555,0.181726,0.314337,0.101799,0.058262,0.683319,0.889533
섬유의복,0.796107,0.896869,1.000000,0.611734,0.721663,0.807659,0.474613,0.583414,0.411954,0.743505,...,0.833035,0.646270,0.265856,0.556390,0.291725,0.515728,0.327945,0.295365,0.754067,0.879833
종이목재,0.350777,0.514889,0.611734,1.000000,0.223564,0.764275,0.636462,0.425570,0.606761,0.280063,...,0.673335,0.828309,0.699391,0.540109,0.573298,0.821419,0.626012,0.689359,0.622696,0.474833
화학,0.956254,0.720400,0.721663,0.223564,1.000000,0.598244,-0.046014,0.794475,0.487278,0.765297,...,0.803611,0.150189,0.164028,0.667911,-0.254153,0.311528,0.197428,0.168571,0.708365,0.921848
의약품,0.711579,0.780014,0.807659,0.764275,0.598244,1.000000,0.342793,0.701258,0.643036,0.618682,...,0.823435,0.748642,0.519600,0.681012,0.369644,0.666025,0.392486,0.567692,0.898190,0.787905
비금속광물,0.009088,0.310595,0.474613,0.636462,-0.046014,0.342793,1.000000,-0.004502,0.156718,0.033645,...,0.255514,0.657169,0.342101,0.173073,0.666155,0.397011,0.293348,0.335578,0.172186,0.196981
철강금속,0.755305,0.479571,0.583414,0.425570,0.794475,0.701258,-0.004502,1.000000,0.846576,0.458625,...,0.822892,0.315843,0.603746,0.919775,-0.073922,0.625279,0.481831,0.616966,0.882905,0.759393
기계,0.461514,0.249604,0.411954,0.606761,0.487278,0.643036,0.156718,0.846576,1.000000,0.136387,...,0.738021,0.383278,0.884151,0.924605,0.124319,0.782659,0.598425,0.862480,0.785014,0.514435
전기전자,0.899321,0.892836,0.743505,0.280063,0.765297,0.618682,0.033645,0.458625,0.136387,1.000000,...,0.646196,0.317519,-0.142280,0.317823,-0.071230,0.153665,0.010885,-0.107219,0.584161,0.851958


In [12]:
model = Prophet()

In [23]:
model.fit(train)

ValueError: Dataframe must have columns "ds" and "y" with the dates and values respectively.